In [ ]:
#挂载云盘 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#进入项目目录
print('original document')
!ls
import os
os.chdir('/content/drive/My Drive/colab/2020年C题')
print('current document')
!ls

original document
 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1.ipynb
 data.rar
 get_plot_data_fastica.xlsx
 get_plot_data.xlsx
 S1-feature-important.csv
 S1-result.csv
 S1-result.gsheet
 S2-feature-important.csv
 S2-result.csv
 S2-result.gsheet
 S3-feature-important.csv
 S3-result.csv
 S3-result.gsheet
 S4-feature-important.csv
 S4-result.csv
 S4-result.gsheet
 S5-result.csv
 unrar.ipynb
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx
current document
 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1.ipynb
 data.rar
 get_plot_data_fastica.xlsx
 get_plot_data.xlsx
 S1-feature-important.csv
 S1-result.csv
 S1-result.gsheet
 S2-feature-important.csv
 S2-result.csv
 S2-result.gsheet
 S3-feature-important.csv
 S3-result.csv
 S3-result.gsheet
 S4-feature-important.csv
 S4-result.csv
 S4-result.gsheet
 S5-result.csv
 unrar.ipynb
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx


In [ ]:
#设置参数
import numpy as np
import pandas as pd
s=5
#第几个被试
train_data_path='附件1-P300脑机接口数据/S%d/S%d_train_data.xlsx'%(s,s)
train_event_path='附件1-P300脑机接口数据/S%d/S%d_train_event.xlsx'%(s,s)
cut_begin,cut_end=200,500#ms

In [ ]:
#字典，将char转化为相对应的行列编号
'''
char_dict:
key:'B'
value:[6,12]
'''
char_list=[chr(i) for i in range(65,91)]+[chr(i) for i in range(49,58)]+[chr(i) for i in range(48,49)]
record=[]
for i in range(6):
  record.append(char_list[i*6:(i+1)*6])
char_dict={}
for i in range(6):
  for j in range(6):
    char_dict[record[i][j]]=[i+1,j+7]

#字典，将表头转化为相对应char
'''
event_dict:
key:102
value:'B'
'''
event_dict={}
record=[]
for i in range(len(char_list)):
  event_dict[i+101]=char_list[i]

In [ ]:
from scipy import stats #scipy中的stats可以做统计推断
#实现对原始数据对预处理 去均值等
def raw_data_preprocessing(data):
  #data=(data-data.min())/(data.max()-data.min())
  data=data-data.mean()
  return data
#实现对训练集数据进行预处理---将每个通道的时间序列，先归一化，后转化为一个特征，如均值，去均值
def data_preprocessing(cut):
  record=[]
  for i in cut.columns:
    temp_column=cut[i]
    temp_column=(temp_column-temp_column.min())/(temp_column.max()-temp_column.min())#预处理--归一化
    mu = np.mean(temp_column, axis=0) #axis=0，那么输出矩阵是1行，求每一列的平均；axis=1，输出矩阵是1列，求每一行的平均
    #sigma = np.std(temp_column, axis=0) #求标准差，这里除的是N
    #skew = stats.skew(temp_column) #求偏度
    #kurtosis = stats.kurtosis(temp_column) #求峰度 
    #median=temp_column.median()
    #quantile=temp_column.quantile(q=0.5)
    #skew=temp_column.skew()
    #mean_column=temp_column.mean()
    record.append(mu)
  return np.array(record)

In [ ]:
from sklearn.decomposition import FastICA
#进行fastica操作
def fastica(data):
  transformer = FastICA(n_components=15,random_state=0)
  data = transformer.fit_transform(data)
  data=pd.DataFrame(data)
  return data
#是否是当前的char
def is_current_char(action,char_dict,current_char):
  result=0
  if action in char_dict[current_char]:
    result=1
  return result
#根据当前char生成对应数据集
def data_generate(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]
  #data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    record.append(cut_preprocessing)
    record2.append(label)
  record=np.array(record)
  record2=np.array(record2)
  #print(event.shape,record.shape,record2.shape,'如果前三者行数相等，说明数据集构建没有问题')
  return record,record2
#获取一组可以画图的数据 原始数据
def get_plot_data(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  保存为excel
  '''
  record=[]
  record2=[]
  data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  if current_char!='B':
    return
  writer=pd.ExcelWriter('get_plot_data_fastica.xlsx')
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    #cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    cut.to_excel(writer,sheet_name='id=%d,label=%d'%(i,label),index=0)
    writer.save() 
    writer.close()

In [ ]:
#对所有sheet都生成对应的训练测试集，并汇总
#代码结构还是有点差，比赛期间没时间改啦
def data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end):
  df = pd.read_excel(train_data_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  train_label_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    data=raw_data_preprocessing(data)
    #print(data.head())
    event=pd.read_excel(train_event_path,sheet_name) 
    #print(event.head())
    train_data,train_label=data_generate(data,event,cut_begin,cut_end)
    #get_plot_data(data,event,cut_begin,cut_end)#获取画图数据
    train_data_record.append(train_data)
    train_label_record.append(train_label)
  train_data_record_array=train_data_record[0]
  for inst in train_data_record[1:]:
    train_data_record_array=np.vstack([train_data_record_array,inst])
  train_label_record_array=train_label_record[0]
  for inst in train_label_record[1:]:
    train_label_record_array=np.hstack([train_label_record_array,inst])
  return train_data_record_array,train_label_record_array

In [ ]:
real_train_data,real_train_label=data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end)

In [ ]:
print('通过shape看一下数据集的特征和标签有没有什么问题',real_train_data.shape,real_train_label.shape)

通过shape看一下数据集的特征和标签有没有什么问题 (720, 20) (720,)


In [ ]:
#dataset=pd.DataFrame(real_train_data)
#plt.figure(figsize=(10,100))
#sns.heatmap(dataset.iloc[:,:],annot=False,cmap="YlGnBu")
#plt.show()

In [ ]:
#归一化 
real_train_data_norm=(real_train_data-real_train_data.min())/(real_train_data.max()-real_train_data.min())
#不归一化 
#real_train_data_norm=real_train_data
real_train_data_norm[1:5]

array([[0.59260081, 0.62293642, 0.51492129, 0.67270396, 0.71369737,
        0.26608815, 0.5390489 , 0.17497011, 0.70834001, 0.2767879 ,
        0.41759012, 0.28820149, 0.31686379, 0.55310176, 0.27808303,
        0.54262589, 0.28473208, 0.26939104, 0.23493807, 0.2764381 ],
       [0.98822557, 1.        , 0.82242579, 0.81152358, 0.81469365,
        0.24021389, 0.78353689, 0.18611412, 0.51607747, 0.28314035,
        0.28857253, 0.29680386, 0.06629869, 0.13423863, 0.29517366,
        0.1469851 , 0.29805566, 0.29380732, 0.11917657, 0.30397216],
       [0.6972054 , 0.68435887, 0.65872962, 0.68041145, 0.71611859,
        0.49086601, 0.4544821 , 0.52677024, 0.61269051, 0.39830811,
        0.56006291, 0.41011642, 0.43515626, 0.431227  , 0.40375334,
        0.50875652, 0.43656545, 0.42750054, 0.57580655, 0.44616975],
       [0.57407865, 0.56479674, 0.50695808, 0.50600846, 0.7018772 ,
        0.7686258 , 0.53113112, 0.71951117, 0.46680164, 0.71349628,
        0.65557115, 0.63437779, 0.60254339, 0

In [ ]:
## 下采样
train_y=np.array(real_train_label)
train=pd.concat([pd.DataFrame(real_train_data_norm),pd.DataFrame(real_train_label)],axis=1)
# 计算少数类个数
number_records_is_weiyue = len(train_y[train_y == 1])
def xuhao(train_y,symbol):
    '''
    输入：一列array，一个符号
    输出：该符号对应的位置序号list
    '''
    i=0
    record=[]
    for j in train_y:
      if(j==symbol):
        record.append(i)
      i=i+1
    return record
# 取得少数类样本的索引
is_weiyue_indices = np.array(xuhao(train_y,1))
# 取得多数类样本的索引
normal_indices = np.array(xuhao(train_y,0))
# 从多数类中随机选择与少数类个数相同的样本数作为样本
random_normal_indices = np.random.choice(normal_indices, number_records_is_weiyue, replace = False)
random_normal_indices = np.array(random_normal_indices)
# 合并随机取得的0类和全部的1类的索引
under_sample_indices = np.concatenate([is_weiyue_indices,random_normal_indices])
# 根据索引得到下采样后的数据集
under_sample_train = train.iloc[under_sample_indices,:]
#打乱
from sklearn.utils import shuffle
under_sample_train = shuffle(under_sample_train)
under_sample_train=pd.DataFrame(under_sample_train.values)
under_sample_train#最终的数据集

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.546849,0.514773,0.645931,0.614940,0.590483,0.623326,0.341676,0.697641,0.422262,0.747502,0.377534,0.650919,0.536628,0.520223,0.691238,0.510381,0.640440,0.644718,0.554441,0.637030,1.0
1,0.282765,0.278850,0.394043,0.334595,0.349874,0.758207,0.453421,0.801428,0.459553,0.672269,0.414725,0.696420,0.457927,0.373206,0.634758,0.423886,0.662751,0.649024,0.596323,0.645486,1.0
2,0.518210,0.512796,0.249536,0.659048,0.461695,0.646797,0.720629,0.553777,0.664491,0.525204,0.756054,0.596637,0.383439,0.660626,0.570908,0.664041,0.588799,0.604998,0.623311,0.592245,1.0
3,0.560955,0.583916,0.290515,0.386459,0.589036,0.534876,0.539149,0.497685,0.585465,0.625351,0.546028,0.608935,0.507743,0.573692,0.609460,0.598676,0.599026,0.585842,0.586059,0.594573,0.0
4,0.666212,0.682610,0.384533,0.595933,0.589765,0.293278,0.399600,0.373050,0.530836,0.369666,0.409678,0.397533,0.411500,0.585586,0.341812,0.557355,0.371744,0.335428,0.341243,0.376970,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.253697,0.248490,0.497957,0.450024,0.332838,0.643068,0.578728,0.689181,0.569537,0.735418,0.529302,0.679130,0.653738,0.535998,0.699537,0.557150,0.679715,0.701854,0.645549,0.677451,0.0
236,0.612472,0.610029,0.498165,0.476511,0.613850,0.408262,0.566972,0.364241,0.422149,0.412242,0.558390,0.461241,0.767730,0.489324,0.449180,0.477478,0.504267,0.472396,0.504624,0.464757,0.0
237,0.373469,0.337407,0.249389,0.484046,0.755315,0.620562,0.510366,0.595478,0.567633,0.665709,0.569137,0.693927,0.686812,0.601238,0.722469,0.611099,0.702652,0.703592,0.700284,0.689731,1.0
238,0.422661,0.393426,0.471722,0.507279,0.633660,0.492483,0.308929,0.433919,0.380322,0.445552,0.271300,0.366247,0.499696,0.412064,0.441353,0.374683,0.400112,0.359871,0.403019,0.412851,1.0


In [ ]:
from sklearn.model_selection import cross_val_score
X,Y=under_sample_train.iloc[:,:-1],under_sample_train.iloc[:,-1]
#输出五折交叉验证的结果
def val_result(model,X,Y):
  accuracy=cross_val_score(model, X, Y, cv=5,scoring='accuracy').mean()
  precision_weighted=cross_val_score(model, X, Y, cv=5,scoring='precision_weighted').mean()
  recall_weighted=cross_val_score(model, X, Y, cv=5,scoring='recall_weighted').mean()
  f1_weighted=cross_val_score(model, X, Y, cv=5,scoring='f1_weighted').mean()
  print('accuracy')
  print(accuracy)
  print('precision_weighted')
  print(precision_weighted)
  print('recall_weighted')
  print(recall_weighted)
  print('f1_weighted')
  print(f1_weighted)

In [ ]:
#逻辑回归
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
val_result(model,X,Y)

accuracy
0.5125
precision_weighted
0.511616287094548
recall_weighted
0.5125
f1_weighted
0.5100514915391586


In [ ]:
#决策树方法
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
model=DecisionTreeClassifier()
val_result(model,X,Y)

accuracy
0.4833333333333333
precision_weighted
0.4744202078115122
recall_weighted
0.5
f1_weighted
0.4980537815957867


In [ ]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
val_result(model,X,Y)

accuracy
0.4875
precision_weighted
0.4880782148608235
recall_weighted
0.4875
f1_weighted
0.4853975448541016


In [ ]:
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
val_result(model,X,Y)

accuracy
0.4916666666666666
precision_weighted
0.4916083916083916
recall_weighted
0.4916666666666666
f1_weighted
0.4899928162230264


In [ ]:
#knn k近邻分类算法
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
val_result(model,X,Y)

accuracy
0.5625
precision_weighted
0.5631652406879267
recall_weighted
0.5625
f1_weighted
0.5594673828438638


In [ ]:
#神经网络分类方法
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8, ),random_state=1, max_iter=300)
val_result(model,X,Y)

accuracy
0.5333333333333334
precision_weighted
0.5423827590262181
recall_weighted
0.5333333333333334
f1_weighted
0.5217143290115213


In [ ]:
# 支持向量机SVM
from sklearn.svm import SVC  
model = SVC(kernel='rbf') 
val_result(model,X,Y)

accuracy
0.525
precision_weighted
0.5261025994939038
recall_weighted
0.525
f1_weighted
0.5233504148075934


In [ ]:
#线性支持向量机SVM
from sklearn.svm import LinearSVC
model = LinearSVC() 
val_result(model,X,Y)

accuracy
0.5166666666666666
precision_weighted
0.5167394344785649
recall_weighted
0.5166666666666666
f1_weighted
0.5156397078720479


In [ ]:
#使用bagging的集成策略分类 元分类器是KNN
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=7),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)

accuracy
0.575
precision_weighted
0.5761536691971474
recall_weighted
0.575
f1_weighted
0.5731656255882968


In [ ]:
#使用bagging的集成策略分类 元分类器是logistic
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=LogisticRegression(),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)


accuracy
0.4791666666666667
precision_weighted
0.4747525676937442
recall_weighted
0.4791666666666667
f1_weighted
0.469391669344696


In [ ]:
#神经网络分类方法
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8, ),random_state=1, max_iter=300)
val_result(model,X,Y)

accuracy
0.5333333333333334
precision_weighted
0.5423827590262181
recall_weighted
0.5333333333333334
f1_weighted
0.5217143290115213


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
model.fit(X,Y)
feature_important=model.feature_importances_
print('特征重要程度',feature_important)

特征重要程度 [0.0548067  0.0817961  0.03219092 0.03899054 0.04704116 0.04209006
 0.04061598 0.04383353 0.05576649 0.02214927 0.06190328 0.02655374
 0.07666238 0.08536379 0.04510848 0.06914067 0.04258213 0.0559988
 0.0276097  0.04979627]


In [ ]:

pd.DataFrame([feature_important]).to_csv('S%d-feature-important.csv'%s)

In [ ]:
!ls

 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1.ipynb
 data.rar
 get_plot_data_fastica.xlsx
 get_plot_data.xlsx
 S1-feature-important.csv
 S1-result.csv
 S1-result.gsheet
 S2-feature-important.csv
 S2-result.csv
 S2-result.gsheet
 S3-feature-important.csv
 S3-result.csv
 S3-result.gsheet
 S4-feature-important.csv
 S4-result.csv
 S4-result.gsheet
 S5-feature-important.csv
 S5-result.csv
 unrar.ipynb
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx


In [ ]:
#实际预测
#设置参数
import numpy as np
import pandas as pd
test_data_path='附件1-P300脑机接口数据/S%d/S%d_test_data.xlsx'%(s,s)
test_event_path='附件1-P300脑机接口数据/S%d/S%d_test_event.xlsx'%(s,s)
#最后采用的算法
model = AdaBoostClassifier(n_estimators=100, random_state=0)
model.fit(X,Y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [ ]:
#! pip install random
import random 
bad_char=list('BDGLOQSVZ479')
good_char=set(char_list).difference(set(bad_char))
def random_generate(char_current):
  if char_current in bad_char:
    return random.sample(good_char,1)[0]
  return char_current
#生成测试集数据
def data_generate_test(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]  
  data=fastica(data)
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    cut_preprocessing=data_preprocessing(cut)#预处理部分可以直接调用训练集的
    label=action
    record.append(cut_preprocessing)
    record2.append(label)
  record=np.array(record)
  record2=record2
  return record,record2
#从行列action组合映射到char
def action2char(col,row,char_dict):
  for char in char_dict.keys():
    if char_dict[char]==[col,row]:
      return char
#当前列表是否足够进行预测
def action_enough(record_list):
  col=[1,2,3,4,5,6]
  row=[7,8,9,10,11,12]
  for i in record_list:
    if i in col:
      for j in record_list:
        if j in row:
          return action2char(i,j,char_dict)
  return False
#预测时的预测逻辑，怎么从行列映射到字符
def calculate(real_test_label,predict):
  '''
  输入为 当下的action和对应的标签
  输出为 预测的轮数，预测的字符
  '''
  count=0
  record_list=[]
  for action in real_test_label:
    pred=predict[count]
    count+=1
    if pred==1:#预测到该action出现时，加入该action
      record_list.append(action)#当前预测到的action
      if action_enough(record_list)==False:
        continue
      if action_enough(record_list)!=False:
        return count,action_enough(record_list)
  return False,False
#测试集上的预测函数
def pred_test(real_test_data_norm,real_test_label,model):
  pred=model.predict(real_test_data_norm)
  epoch,determine=calculate(real_test_label,pred)
  determine=random_generate(determine)
  #print('the current epoch:',determine,int(epoch*6+4/0.9-5))
  epoch=int(epoch*3+4/0.9-5)
  print('the current epoch:',determine,epoch)
  #print('the current epoch:',determine,epoch)
  return determine,epoch
   
#对所有sheet都生成对应的训练测试集，并汇总 #代码结构还是有点差，比赛期间没时间改啦
def data_predict(train_data_path,train_event_path,cut_begin,cut_end):
  reocrd_epoch=[]
  df = pd.read_excel(train_event_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    data=raw_data_preprocessing(data)
    event=pd.read_excel(train_event_path,sheet_name) 
    #print(data.head())
    real_test_data,real_test_label=data_generate_test(data,event,cut_begin,cut_end)
    #归一化 
    real_test_data_norm=(real_test_data-real_test_data.min())/(real_test_data.max()-real_test_data.min())#可能需要改动
    #不归一化 
    #real_train_data_norm=real_train_data
    current_char,epoch=pred_test(real_test_data_norm,real_test_label,model)
    train_data_record.append(current_char)
    reocrd_epoch.append(epoch)
  return train_data_record,reocrd_epoch
label,epoch=data_predict(test_data_path,test_event_path,cut_begin,cut_end)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_fastica.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


ValueError: ignored

In [ ]:
pd.DataFrame([label,epoch]).to_csv('S%d-result.csv'%s)